<a href="https://colab.research.google.com/github/tiaaburton/Milestone-II-Machine-Learning-Project/blob/main/Playground_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
! pip install pyspark

In [40]:
! pip install pyspark[pandas-on-spark] plotly

In [42]:
import pandas as pd
import numpy as np
import re
import os
import sys

from google.cloud import bigquery
import pyarrow
from datetime import datetime
from tqdm import tqdm

from oauth2client.client import GoogleCredentials
from google.colab import auth, files

import ast
import json
import re
import random

import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import pyspark.ml
from pyspark.ml import *



In [27]:
RANDOM_SEED = 655

In [43]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Binary Buyer Prediction") \
    .getOrCreate()

In [ ]:
sample = spark.read.csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/processed_file.csv', header=True).withColumnRenamed('_c0','index')

In [ ]:
# sample.head()

Row(index='0', date='20160801', geoNetwork="{'continent': 'Americas', 'subContinent': 'Northern America', 'country': 'United States', 'region': 'not available in demo dataset', 'metro': 'not available in demo dataset', 'city': 'not available in demo dataset', 'cityId': 'not available in demo dataset', 'networkDomain': 'unknown.unknown', 'latitude': 'not available in demo dataset', 'longitude': 'not available in demo dataset', 'networkLocation': 'not available in demo dataset'}", channelGrouping='Social', fullVisitorId='7305625498291809599', socialEngagementType='Not Socially Engaged', trafficSource="{'referralPath': '/yt/about/', 'campaign': '(not set)', 'source': 'youtube.com', 'medium': 'referral', 'keyword': None, 'adContent': None, 'adwordsClickInfo': {'campaignId': None, 'adGroupId': None, 'creativeId': None, 'criteriaId': None, 'page': None, 'slot': None, 'criteriaParameters': 'not available in demo dataset', 'gclId': None, 'customerId': None, 'adNetworkType': None, 'targetingCri

In [ ]:
# sample.printSchema()

In [ ]:
from collections import Counter
core = [col for col in sample.columns if '.' not in col]
additional = [col for col in sample.columns if '.' in col]
second = []
third = []

for col in additional:
  count = Counter(col)
  if count['.'] == 1:
    second.append(col)
  elif count['.'] == 2:
    third.append(col)

In [ ]:
# second.sort()
# third.sort()

In [ ]:
# input = sample.select(core)

In [ ]:
# for ls in [second, third]:
#   for col in ls:
#     split = col.split('.')
#     input = input.withColumn(col, get_json_object(split[0], '$.'+split[1]))

In [96]:
# import json
# from pyspark.sql.types import *

# split = second[0].strip().split('.')

# def parse_json(array_str):
#   json_obj = json.loads(array_str)
#   for item in json_obj:
#     yield (item[split[1]])
    


# json_schema = ArrayType(StructType([StructField(split[1], StringType(), nullable=True)]))
# udf_parse_json = udf(lambda str, col: parse_json(str, col), json_schema)

# # input.withColumn('test'+split[1],from_json(split[0], split[1])).select('test'+split[1]).take(10)


In [ ]:
# input.take(50)

In [4]:
columns = []
with open('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Notebooks/final_columns.txt') as f:
  for ln in f.readlines():
    saved = ln.strip().split('\'')
    columns.append(saved[1])
print(columns)

['date', 'totals', 'fullVisitorId', 'clientId', 'totals.visits', 'totals.hits', 'totals.pageviews', 'totals.timeOnSite', 'totals.bounces', 'totals.transactions', 'totals.newVisits', 'totals.screenviews', 'totals.uniqueScreenviews', 'totals.timeOnScreen', 'totals.totalTransactionRevenue', 'hits.type', 'hits', 'channelGrouping', 'socialEngagementType', 'geoNetwork.country', 'trafficSource.source', 'trafficSource.medium', 'trafficSource.isTrueDirect', 'device.browser', 'device.operatingSystem', 'device.deviceCategory', 'hits.hour', 'hits.minute', 'hits.transaction', 'hits.item', 'hits.dataSource', 'hits.transaction.transactionRevenue', 'hits.item.productName', 'hits.item.productCategory', 'hits.item.itemRevenue', 'hits.appInfo.screenDepth', 'hits.eCommerceAction.action_type']


In [5]:
processed = pd.read_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/processed_file.csv',index_col=0)

In [144]:
processed.head()

,date,geoNetwork,channelGrouping,fullVisitorId,socialEngagementType,trafficSource,device,totals,hits,hits.type,...,geoNetwork.country,device.operatingSystem,device.deviceCategory,totals.pageviews,hits.appInfo.screenDepth,hits.eCommerceAction.action_type,hits.transaction.transactionRevenue,hits.item.productName,hits.item.itemRevenue,hits.item.productCategory
0,20160801,"{'continent': 'Americas', 'subContinent': 'Nor...",Social,7305625498291809599,Not Socially Engaged,"{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'visits': 1, 'hits': 4, 'pageviews': 2, 'time...","[{'hitNumber': 1, 'time': 0, 'hour': 15, 'minu...","['PAGE', 'EVENT', 'PAGE', 'EVENT']",...,United States,Windows,desktop,2.0,"['0', '0', '0', '0']","['0', '0', '0', '0']","[None, None]","[None, None]","[None, None]","[None, None]"
1,20160801,"{'continent': 'Americas', 'subContinent': 'Nor...",Social,5270776363703942229,Not Socially Engaged,"{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...","['PAGE', 'PAGE', 'PAGE', 'PAGE']",...,United States,Windows,desktop,4.0,"['0', '0', '0', '0']","['0', '0', '0', '0']","[None, None, None]","[None, None, None]","[None, None, None]","[None, None, None]"
2,20160801,"{'continent': 'Americas', 'subContinent': 'Nor...",Social,5345363805648300707,Not Socially Engaged,"{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'visits': 1, 'hits': 7, 'pageviews': 7, 'time...","[{'hitNumber': 1, 'time': 0, 'hour': 15, 'minu...","['PAGE', 'PAGE', 'PAGE', 'PAGE', 'PAGE', 'PAGE...",...,United States,Windows,desktop,7.0,"['0', '0', '0', '0', '0', '0', '0']","['0', '0', '0', '0', '0', '0', '0']","[None, None, None, None, None, None]","[None, None, None, None, None, None]","[None, None, None, None, None, None]","[None, None, None, None, None, None]"
3,20160801,"{'continent': 'Americas', 'subContinent': 'Nor...",Social,5140493641776291415,Not Socially Engaged,"{'referralPath': '/lsr.php', 'campaign': '(not...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",['PAGE'],...,United States,Macintosh,desktop,1.0,['0'],['0'],[None],[None],[None],[None]
4,20160801,"{'continent': 'Asia', 'subContinent': 'Souther...",Social,6137206948447331806,Not Socially Engaged,"{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'visits': 1, 'hits': 20, 'pageviews': 14, 'ti...","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...","['PAGE', 'PAGE', 'PAGE', 'PAGE', 'EVENT', 'PAG...",...,India,Chrome OS,desktop,14.0,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '1', '2', '1', '0', '0', ...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non..."


In [110]:
processed.columns

Index(['date', 'geoNetwork', 'channelGrouping', 'fullVisitorId',
       'socialEngagementType', 'trafficSource', 'device', 'totals', 'hits',
       'hits.type', 'hits.hour', 'hits.minute', 'hits.transaction',
       'hits.item', 'hits.dataSource', 'hits.appInfo', 'hits.eCommerceAction',
       'trafficSource.source', 'totals.visits', 'totals.screenviews',
       'device.browser', 'totals.totalTransactionRevenue', 'totals.bounces',
       'totals.uniqueScreenviews', 'totals.newVisits',
       'trafficSource.isTrueDirect', 'trafficSource.medium', 'totals.hits',
       'totals.transactions', 'totals.timeOnScreen', 'totals.timeOnSite',
       'geoNetwork.country', 'device.operatingSystem', 'device.deviceCategory',
       'totals.pageviews', 'hits.appInfo.screenDepth',
       'hits.eCommerceAction.action_type',
       'hits.transaction.transactionRevenue', 'hits.item.productName',
       'hits.item.itemRevenue', 'hits.item.productCategory'],
      dtype='object')

In [145]:
processed.describe()

,date,fullVisitorId,totals.visits,totals.screenviews,totals.totalTransactionRevenue,totals.bounces,totals.uniqueScreenviews,totals.newVisits,totals.hits,totals.transactions,totals.timeOnScreen,totals.timeOnSite,totals.pageviews
count,3.572500e+04,3.572500e+04,35725.0,0.0,3.630000e+02,18244.0,0.0,28977.0,35725.000000,369.000000,0.0,17447.000000,35718.000000
mean,2.016099e+07,4.537203e+18,1.0,NaN,1.565480e+08,1.0,NaN,1.0,4.740770,1.054201,NaN,242.206626,3.883980
std,1.128160e+02,3.077969e+18,0.0,NaN,2.527672e+08,0.0,NaN,0.0,11.530412,0.308027,NaN,483.392041,8.217082
min,2.016080e+07,3.347106e+13,1.0,NaN,3.560000e+06,1.0,NaN,1.0,1.000000,1.000000,NaN,1.000000,1.000000
25%,2.016091e+07,1.624230e+18,1.0,NaN,3.173500e+07,1.0,NaN,1.0,1.000000,1.000000,NaN,33.000000,1.000000
50%,2.016101e+07,4.444965e+18,1.0,NaN,6.399000e+07,1.0,NaN,1.0,1.000000,1.000000,NaN,72.000000,1.000000
75%,2.016111e+07,7.227935e+18,1.0,NaN,1.496200e+08,1.0,NaN,1.0,4.000000,1.000000,NaN,219.000000,3.000000
max,2.016113e+07,9.999397e+18,1.0,NaN,2.213380e+09,1.0,NaN,1.0,500.000000,4.000000,NaN,11094.000000,469.000000


In [6]:
processed = processed.drop(['totals.screenviews','totals.uniqueScreenviews','totals.timeOnScreen',
                            'device','totals','hits','hits.type','trafficSource','geoNetwork'],axis=1)

In [81]:
processed.head(2)

,date,channelGrouping,fullVisitorId,socialEngagementType,hits.hour,hits.minute,hits.transaction,hits.item,hits.dataSource,hits.appInfo,...,geoNetwork.country,device.operatingSystem,device.deviceCategory,totals.pageviews,hits.appInfo.screenDepth,hits.eCommerceAction.action_type,hits.transaction.transactionRevenue,hits.item.productName,hits.item.itemRevenue,hits.item.productCategory
0,20160801,Social,7305625498291809599,Not Socially Engaged,"[15, 15, 15, 15]","[33, 34, 35, 35]","[None, None, {'transactionId': None, 'transact...","[None, None, {'transactionId': None, 'productN...","[None, None, None, None]","[{'name': None, 'version': None, 'id': None, '...",...,United States,Windows,desktop,2.0,"['0', '0', '0', '0']","['0', '0', '0', '0']","[None, None]","[None, None]","[None, None]","[None, None]"
1,20160801,Social,5270776363703942229,Not Socially Engaged,"[13, 13, 13, 13]","[42, 42, 43, 43]","[None, {'transactionId': None, 'transactionRev...","[None, {'transactionId': None, 'productName': ...","[None, None, None, None]","[{'name': None, 'version': None, 'id': None, '...",...,United States,Windows,desktop,4.0,"['0', '0', '0', '0']","['0', '0', '0', '0']","[None, None, None]","[None, None, None]","[None, None, None]","[None, None, None]"


In [18]:
train = processed[processed.columns[~processed.columns.isin(['totals.transactions'])]]
train = train[['date','fullVisitorId','totals.visits','totals.totalTransactionRevenue',
               'totals.bounces','totals.newVisits','totals.hits','totals.timeOnSite',
               'totals.pageviews']]
               
target = processed[processed.columns[processed.columns.isin(['totals.transactions'])]]
target = pd.Categorical(target.iloc[:,0])
target = pd.Series(target.add_categories(0.0).fillna(0.0))

In [76]:
train["totals.visits"]#.corr()

array([1])

In [85]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import recall_score

import pyspark.pandas as ps
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.feature import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [55]:
spark.newSession()

In [12]:
lst = ['hits.hour', 'hits.minute', 'hits.transaction','hits.item','hits.dataSource','hits.appInfo']
for l in lst:
  train[l] = train[l].apply(lambda x: list(x[1:-1]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
train = train.fillna(0)
train.dtypes, target.dtype

(date                                int64
 fullVisitorId                      uint64
 totals.visits                       int64
 totals.totalTransactionRevenue    float64
 totals.bounces                    float64
 totals.newVisits                  float64
 totals.hits                         int64
 totals.timeOnSite                 float64
 totals.pageviews                  float64
 dtype: object,
 CategoricalDtype(categories=[1.0, 2.0, 3.0, 4.0, 0.0], ordered=False))

In [36]:
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=.3, stratify=target)

In [37]:
dum_clf = DummyClassifier(strategy='stratified', random_state=RANDOM_SEED)
lr = LogisticRegression(solver='lbfgs', max_iter=40000)
dtree = DecisionTreeClassifier(criterion='entropy',random_state=RANDOM_SEED)

In [82]:
X_train_spark = spark.createDataFrame(X_train)
X_val_spark = spark.createDataFrame(X_val)
y_train_spark = ps.from_pandas(y_train)
y_val_spark = ps.from_pandas(y_val)

In [94]:
train_spark = processed[['date','fullVisitorId','totals.visits','totals.totalTransactionRevenue',
               'totals.bounces','totals.newVisits','totals.hits','totals.timeOnSite',
               'totals.pageviews', 'totals.transactions']]

train_spark = spark.createDataFrame(train_spark).withColumn('labels', col('`totals.transactions`'))

In [126]:
spark_lr = LogisticRegression(maxIter=100000, featuresCol='features',labelCol='labels')
paramGrid = ParamGridBuilder()\
    .addGrid(spark_lr.regParam, [0.1, 0.01])\
    .build()

cols = ['`'+col+'`' if (col != 'totals.transactions' and '.' in col) else col for col in train_spark.columns]
cols.remove('totals.transactions')

vec = VectorAssembler(inputCols=cols, outputCol='features',handleInvalid='keep')

pipeline = Pipeline(stages=[vec, spark_lr])
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=2)

In [128]:
crossval.fit(train_spark)

IllegalArgumentException: ignored